In [2]:
import numpy as np
import pandas as pd
import datetime
import re
from os import path
import json

In [4]:
with open(path.expanduser("~") + "/sys_specific.json") as file: 
    params = json.load(file)
_HOME_ = params["home"] + "/PHD"

In [5]:
def remove_additional_white_spaces(string): 
    return " ".join(string.strip().split())

In [6]:
phyto_abund_raw = pd.read_csv(_HOME_ + "/MSFD/Data/Modulo1/phyto_abund_raw.csv")
phyto_abund_raw.rename(columns={"NationalStationID" : "id", "Gruppo" : "Group"}, inplace=True)
phyto_abund_raw["id"]  = phyto_abund_raw["id"].apply(lambda x: x.split(".")[0])
dates = [
    datetime.date(year=year, month=month, day=day) 
    for year, month, day in phyto_abund_raw.loc[:, ["Year", "Month", "Day"]].to_numpy(np.int16)    
]
phyto_abund_raw.drop(columns = ["Autore", "Year", "Month", "Day"], inplace=True)
phyto_abund_raw["Date"] = dates
phyto_abund_raw.loc[phyto_abund_raw["id"] == '0004-M000200_TR03', "id"] = '0004-MS00200_TR03'
phyto_abund_raw.loc[phyto_abund_raw["id"] == '0004-M000200_TR06', "id"] = '0004-MS00200_TR06'
phyto_abund_raw.loc[phyto_abund_raw["id"] == '0004-M000200_TR12', "id"] = '0004-MS00200_TR12'

/tmp/ipykernel_10637/3992562802.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  phyto_abund_raw = pd.read_csv(_HOME_ + "/MSFD/Data/Modulo1/phyto_abund_raw.csv")


In [7]:
phyto_abund_raw["Taxon"] = phyto_abund_raw["Taxon"].astype(str)

In [8]:
phyto_abund_raw.dropna(subset=["Num_cell_l"], inplace=True)

In [12]:
df_lat_long_stations = pd.read_csv(_HOME_ + "/ISPRA_20152017_Analysis/station_locations.csv")

In [14]:
df_lat_long_stations

,id,Longitude,Latitude
0,1_ABR_GU04,14.0418,42.7609
1,1_ABR_GU05,14.1072,42.7744
2,1_ABR_GU06,14.2380,42.8012
3,1_ABR_VA13,14.7225,42.2216
4,1_ABR_VA14,14.7609,42.2627
...,...,...,...
157,50720,12.4172,45.0979
158,60530,12.4504,45.2852
159,60720,12.4816,45.1220
160,70530,12.5920,45.2780


In [13]:
phyto_abund_raw = phyto_abund_raw.merge(df_lat_long_stations[["id", "Region", "Longitude", "Latitude", "Closest_coast"]], how = "left", left_on="id", right_on="id")

KeyError: "['Region', 'Closest_coast'] not in index"

In [9]:
phyto_abund_raw["Taxon"] = np.where(phyto_abund_raw["Taxon"].isna(), phyto_abund_raw["NuovoTaxon"], phyto_abund_raw["Taxon"])
phyto_abund_raw.loc[~phyto_abund_raw["Taxon"].isna(), "Taxon"] = phyto_abund_raw.loc[~phyto_abund_raw["Taxon"].isna(), "Taxon"].apply(remove_additional_white_spaces)

In [10]:
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Trattasi di Chlorophyceae indet", na = False),"Taxon"] = "Chlorophyceae indet."
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Trattasi di Chlorophyceae indet", na = False), "Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Cryptophyceae", na = False),"Taxon"] = "Cryptophyceae indet."
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Cryptophyceae", na = False), "Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Leptocylindrus danicus complex sp.p.", na = False), "Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Leptocylindrus sp.p. o Pseudoguinardia recta", na = False), "Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Leptocylindrus cf. convexus", na = False), "Taxon"] = "Leptocylindrus convexus"
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Leptocylindrus cf. convexus", na = False), "Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Gymnodiniales", na = False), "Taxon"] = "Gymnodiniales indet."
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Gymnodiniales", na = False), "Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Karenia", na = False), "Taxon"] = "Karenia spp."
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Karenia", na = False), "Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Karlodinium", na = False), "Taxon"] = "Karlodinium spp."
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Karlodinium", na = False), "Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("cf. polyedricum", na = False), "Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("inf.", na = False, regex = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("sup.", na = False, regex = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("cf. cariense", na = False, regex = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Pseudoguinardia recta", na = False),"Taxon"] =  "Pseudoguinardia spp."
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Pseudoguinardia recta", na = False),"Remarks"] =  None
phyto_abund_raw.loc[(phyto_abund_raw["Remarks"].str.contains("classe Cyanophyceae", na = False)) & phyto_abund_raw["Taxon"].isna(), "Taxon"] = "Cyanophyceae indet."
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("classe Cyanophyceae", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Incluse nella Classe delle Mediophyceae", na = False),"Remarks"] = None 
phyto_abund_raw.loc[phyto_abund_raw["Remarks"] == "n","Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Naviculaceae indet", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Pennales", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("in sostituzione di Leptocylindrus minimus", na = False),"Remarks"] = None 
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Pronoctiluca cf. pelagica", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Protoperidinium cf. cerasus", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("certezza della specie", na = False),"Taxon"] = phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("certezza della specie", na = False),"Taxon"].apply(lambda x: x.split(" ")[0] + " spp.")
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("certezza della specie", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("si  riferisce", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Protoperidinium cf. steinii", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Proboscia sp", na = False),"Taxon"] = "Proboscia sp." 
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Proboscia sp", na = False),"Remarks"] = None 
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("SP", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Protoperidinium", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Probabilmente appartenente ", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Pseudo-nitzschia sp", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains(" cf\.", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Rhabdolithes claviger", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Solenicola setigera", na = False),"Remarks"] = None
phyto_abund_raw.loc[phyto_abund_raw["Remarks"].str.contains("Phalacroma rotundatum", na = False),"Remarks"] = None

In [11]:
phyto_abund_raw.loc[phyto_abund_raw["Taxon"] == "nan","Taxon"] = phyto_abund_raw.loc[phyto_abund_raw["Taxon"] == "nan","Group"]

In [12]:
phyto_abund_raw.loc[phyto_abund_raw["Taxon"].isna(), :]

,id,Time,SampleDepth,Group,Taxon,NuovoTaxon,Num_cell_l,file_name,Remarks,Date,Region,Longitude,Latitude,Closest_coast


In [13]:
phyto_abund_raw.loc[phyto_abund_raw["Taxon"].str.contains("Nan"), :]

,id,Time,SampleDepth,Group,Taxon,NuovoTaxon,Num_cell_l,file_name,Remarks,Date,Region,Longitude,Latitude,Closest_coast


In [14]:
phyto_abund_raw["Taxon"] = phyto_abund_raw["Taxon"].apply(str.capitalize)

In [15]:
phyto_abund_raw.loc[phyto_abund_raw["Taxon"].isna(), :]

,id,Time,SampleDepth,Group,Taxon,NuovoTaxon,Num_cell_l,file_name,Remarks,Date,Region,Longitude,Latitude,Closest_coast


In [16]:
phyto_abund_raw.loc[phyto_abund_raw["Taxon"].str.contains("Nan"), "Taxon"] = phyto_abund_raw.loc[phyto_abund_raw["Taxon"].str.contains("Nan"), "NuovoTaxon"]

In [17]:
phyto_abund_raw.loc[phyto_abund_raw["Taxon"] == "Incertae sedis", "Taxon"] = "Other phytoplankton"

In [18]:
partial_corrections = { 
 "Phalachroma" : "Phalacroma", 
 "Protoperidium": "Protoperidinium",
 "Nefroselmis" : "Nephroselmis",
 "Minuscola" : "Minuscula",
 "Tecati" : "Tecatia", 
 "Heteroleiblenia" : "Heteroleibleinia", 
 "Haloshpaera" : "Halosphaera", 
 "Distephanus" : "Distephanopsis", 
 "Talassionema" : "Thalassionema", 
 "Hetrocapsa" : "Heterocapsa", 
 "Pyramonas" : "Pyramimonas", 
 "Nitzchia" : "Nitzschia", 
 "Chaetocheros" : "Chaetoceros", 
 "Rhodomonascf\." : "Rhodomonas", 
 "Syrachosphaera" : "Syracosphaera", 
 "Indeterminat[^\s|a-z]" : "Indeterminato", 
 "Gunardia" : "Guinardia", 
 "Oxitoxum" : "Oxytoxum", 
 "Altro_fitoplanctonerminati" : "Other phytoplankton", 
 "Mesoporus" : "Mesoporos", 
 "Schuetiella" : "Schuettiella", 
 "Chatoceros" : "Chaetoceros", 
 "Dicosphaera" : "Discosphaera", 
 "Prorocenrum" : "Prorocentrum", 
 "Chrysocromulina" : "Chrysochromulina", 
 "Dictyoca" : "Dictyocha", 
 "Mesoporo" : "Mesoporos", 
 "Octastis": "Octactis", 
 "Ceratum" : "Ceratium", 
 "Schripsiella" : "Scrippsiella", 
 "Ceratulina" : "Cerataulina", 
 "Guanardia" : "Guinardia", 
 "Leptocilindrus" : "Leptocylindrus", 
 "Cilindrotheca" : "Cylindrotheca", 
 "Hemialus" : "Hemiaulus", 
 "Thalassionemasp\." : "Thalassionema sp.", 
 "Asterinellopsis" : "Asterionellopsis", 
 "Dactysolen" : "Dactyliosolen",
 "Dactylosolen" : "Dactyliosolen",
 "Psudosolenia" : "Pseudosolenia", 
 "Amphididium" : "Amphidinium", 
 "Oxyoxum" : "Oxytoxum", 
 "Crysophyceae" : "Chrysophyceae", 
 "Hasleasp" : "Haslea sp.", 
 "Gyrodinum" : "Gyrodinium", 
 "Gymnodium" : "Gymnodinium", 
 "Mesopros" : "Mesoporos", 
 "Hemialius" : "Hemiaulus", 
 "Gonylaux" : "Gonyaulax", 
 "Proprocentrum" : "Prorocentrum", 
 "Protopteridinium" : "Protoperidinium", 
 "Kofodinium" : "Kofoidinium",
 "Protopterodinium" : "Protoperidinium", 
 "Chaetocerus" : "Chaetoceros", 
 "Rhabdosphera" : "Rhabdosphaera", 
 "Girosigma" : "Gyrosigma", 
 "Protoperidinuum" : "Protoperidinium", 
 "Alexandrinum" : "Alexandrium", 
 "Brachydinium" : "Brachidinium",
 "Prococystis" : "Protocystis",
 "Cheatoceros" : "Chaetoceros",
 "Dinoflagellati" : "Dinoflagellata",
 "fraudolenta" : "fraudulenta",
 "Pseudo nitzschia" : "Pseudo-nitzschia", 
 "pseudelicatissima" : "pseudodelicatissima", 
 "Pseudonitzschia" : "Pseudo-nitzschia",
 "Pseudo-nitzscha" : "Pseudo-nitzschia", 
 "peruvianos" : "peruvianus", 
 "crvisetus" : "curvisetus", 
 "Mesopoross" : "Mesoporos", 
 "Tetraedon" : "Tetraedron"
}


complete_corrections = {
 "Pseudonitzschia" : "Pseudo-nitzschia", 
"Gunardia striata" : "Guinardia striata", 
"Guanardia striata" : "Guinardia striata", 
"Gymnodinium wulfii" : "Gymnodinium wulffii",
"Altro fitoplancton" : "Other phytoplankton", 
"Gyrodinium fusiform" : "Gyrodinium fusiforme",
"Gyrodinium fusiformis." : "Gyrodinium fusiforme",
"Altro fitoplancton indet." : "Other phytoplankton",
"Hemialius hauckii" : "Hemiaulus hauckii", 
"Hemiaulus haukii" : "Hemiaulus hauckii",
"Hemiaulus senensis" : "Hemiaulus sinensis",
"Hemiaulus seniensis" :  "Hemiaulus sinensis",
"Hemiaulus siniensis" : "Hemiaulus sinensis",
"Heterocapsa rotondata" : "Heterocapsa rotundata",
"Karenia papillionacea" : "Karenia papilionacea",
"Karenia papillonacea" : "Karenia papilionacea", 
"Kofodinium sp." : "Kofoidinium sp.",
"Lauderia annullata" : "Lauderia annulata",
"Leptocylindrus cf. . convexus" : "Leptocylindrus convexus", 
"Leptocylindrus miimus" : "Leptocylindrus minimus",
"Leptocylindrus minimum" : "Leptocylindrus minimus",
"Mesoporus adriaticus" : "Mesoporos adriaticus",
"Mesoporus spp." : "Mesoporos spp.",
"Mesopros sp." : "Mesoporos sp.",
"Mesoporo" : "Mesoporo sp.",
"Minuscola bipes" : "Minuscula bipes",
"Navicula" : "Navicula spp.",
"Nitzchia sp." : "Nitzschia sp.",
"Prorocentrum compressus" : "Prorocentrum compressum",
"Prorocentrum sp. inf 20 µm" : "Prorocentrum sp. (ø inf. 20µm)",
"Prorocentrum sp. sup 20 µm" : "Prorocentrum sp. (ø sup. 20µm)",
"Prorocentrum spp. sup 20 µm" : "Prorocentrum spp. (ø sup. 20µm)",
"Group" : "Other phytoplankton",
"Protoperidinium inf 20 µm" : "Protoperidinium indet. (ø inf. 20µm)",
"Protoperidinium o pentapharsodinium" : "Peridiniales indet.",
"Protopteridinium steini" : "Protoperidinium steinii",
"Protopterodinium steini" : "Protoperidinium steinii",
"Pseudosolenia calcar avis" : "Pseudosolenia calcar-avis",
"Psudosolenia calcar-avis" : "Pseudosolenia calcar-avis",
"Rhizosolenia calcar avis" : "Rhizosolenia calcar-avis",
"Rhizosolenia clevei var communis" : "Rhizosolenia clevei var. communis",
"Thalassionema fraunfeldii" : "Thalassionema frauenfeldii",
"Thalassionema nitzschiodies" : "Thalassionema nitzschioides",
"Thalassionema nitzschoides" : "Thalassionema nitzschioides",
"Syracosphaera pulcra" : "Syracosphaera pulchra",
"Indeterminat inon flagellati sup 20 µm" : "Non flagellates indet. (ø sup. 20µm)", 
"Altro fitoplancton indet. (ø inf. 20µm)" : "Other phytoplankton (ø inf. 20µm)",
"Altro fitoplancton indet. (ø sup. 20µm)" : "Other phytoplankton (ø sup. 20µm)", 
"Altro_fitoplanctonerminati inf 20 µm" : "Other phytoplankton (ø sup. 20µm)", 
"Altro_fitoplanctonerminati non flagellati sup 20 µm" : "Non flagellates indet. (ø sup. 20µm)", 
"Asteromphaluscf. parvulus" : "Asteromphalus parvulus",
"Ceratium fusum" : "Ceratium fusus",
"Asteromphaluscf. sarcophagus" : "Asteromphalus sarcophagus",
"Altro_fitoplancton inf 20 µm" : "Other phytoplankton (ø inf. 20µm)", 
"Altro_fitoplancton non flagellati sup 20 µm" : "Non flagellates indet. (ø sup. 20µm)", 
"Bacillariophyceae" : "Bacillariophyceae indet.", 
"Bacillariophyceae < 20" : "Bacillariophyceae indet. (ø inf. 20µm)", 
"Bacillariophyceae indet. < 20" : "Bacillariophyceae indet. (ø inf. 20µm)",
"Bacillariophyceae indet. < 20um" : "Bacillariophyceae indet. (ø inf. 20µm)",
"Bacillariophyceae inf 20 micron" : "Bacillariophyceae indet. (ø inf. 20µm)",
"Bacillariophyceae sup 20 micron" : "Bacillariophyceae indet. (ø sup. 20µm)", 
"Bacillariophyta" : "Bacillariophyta indet.",
"Bacillariopyta inf 20 micron" : "Bacillariophyta indet. (ø inf. 20µm)",
"Bacillariophyta inf 20 micron" : "Bacillariophyta indet. (ø inf. 20µm)", 
"Bacillariophyta inf 20 µm" : "Bacillariophyta indet. (ø inf. 20µm)", 
"Bacillariophyta inf. 20 micron" : "Bacillariophyta indet. (ø inf. 20µm)",
"Bacillariophyta inf. 20 µm" : "Bacillariophyta indet. (ø inf. 20µm)",
"Bacillariophyta sup 20 micon" : "Bacillariophyta indet. (ø sup. 20µm)",
"Bacillariophyta sup 20 micron" : "Bacillariophyta indet. (ø sup. 20µm)",
"Bacillariophyta sup 20 µm" : "Bacillariophyta indet. (ø sup. 20µm)",
"Bacillariophyta sup. 20 micron" : "Bacillariophyta indet. (ø sup. 20µm)",
"Bacillariophyta sup. 20 µm": "Bacillariophyta indet. (ø sup. 20µm)",
"Bacillariophyta indet. (ø sup. 20µm)" :"Bacillariophyta indet. (ø sup. 20µm)", 
"Cf. . amphidinium spp." : "Amphidinium spp.",
"Chaetoceros anastamousus" : "Chaetoceros anastomosans",
"Chaetoceros anastemousus" : "Chaetoceros anastomosans",
"Chaetoceros anastomonas" : "Chaetoceros anastomosans",
"Chaetoceros anastomosous" : "Chaetoceros anastomosans",
"Chaetoceros anastomosus" : "Chaetoceros anastomosans",
"Chaetoceros anastomousus" : "Chaetoceros anastomosans",
"Chaetoceros anastoumosus" : "Chaetoceros anastomosans", 
"Chlorodendrophyceae" : "Chlorodendrophyceae indet.", 
"Chlorophyceae" : "Chlorophyceae indet.", 
"Chlorophyta" : "Chlorophyta indet.", 
"Chriptophyceae" : "Cryptophyceae indet.", 
"Chroococcales" : "Chroococcales indet.", 
"Chrysochromulina" : "Chrysochromulina sp.", 
"Chrysocromulina" : "Chrysochromulina sp.", 
"Chrysophyceae" : "Chrysophyceae indet.",
"Chrysophyceae indet" : "Chrysophyceae indet.", 
"Cianobatteri filamentosi" : "Cyanobacteria",
"Cianobatteri non filamentosi" : "Cyanobacteria", 
"Cianophyceae indet" : "Cyanophyceae indet.", 
"Cisti di protoperidinium sp." : "Protoperidinium sp.", 
"Coccolithophyceae" : "Coccolithophyceae indet.",
"Coccolithophyceae indet. < 20" : "Coccolithophyceae indet. (ø inf. 20µm)",
"Coccolithopyceae" : "Coccolithophyceae indet.",
"Coccolithphyceae" : "Coccolithophyceae indet.",
"Coccolitophyceae" : "Coccolithophyceae indet.",
"Coccolitophyceae indet." : "Coccolithophyceae indet.",
"Coccolitophyceae sup 20 µm" : "Coccolithophyceae indet. (ø sup. 20µm)",
"Coscinodiscophyceae" : "Coscinodiscophyceae indet.", 
"Cryptophyceae" : "Cryptophyceae indet.",
"Cyanophyceae" : "Cyanophyceae indet.",
"Cyanophyceae indet" : "Cyanophyceae indet.",
"Cyanophyceae indet filamenti" : "Cyanophyceae indet.", 
"Diatoma" : "Diatoma sp.",
"Diatoma s. p." : "Diatoma sp.",
"Diatoma s. p. p." : "Diatoma spp.",
"Diatomee indet" : "Diatoms indet.",
"Dictyochophyceae" : "Dictyochophyceae indet.", 
"Dinoflagellata" : "Dinoflagellata indet.",
"Dinophyceae" : "Dinophyceae indet.",
"Dinophyceae indet (ø sup. 20µm)" : "Dinophyceae indet. (ø sup. 20µm)",
"Dinophyceae inf 20 micron" : "Dinophyceae indet. (ø inf. 20µm)",
"Dinophyceae inferiori 20 micron" : "Dinophyceae indet. (ø inf. 20µm)",
"Dinophyceae sup 20 micron" : "Dinophyceae indet. (ø sup. 20µm)",
"Dinophyceae sup 20 µm" : "Dinophyceae indet. (ø sup. 20µm)",
"Dinophyceae sup. 20 micron" : "Dinophyceae indet. (ø sup. 20µm)",
"Dinophyceae superiori 20 micron" : "Dinophyceae indet. (ø sup. 20µm)", 
"Euglenophyceae" : "Euglenophyceae indet.",
"Euglenophyceae indet. < 20 µm" : "Euglenophyceae indet. (ø inf. 20µm)",
"Euglenophyceae indet. > 20 µm" : "Euglenophyceae indet. (ø sup. 20µm)",
"Eutreptiaceae" : "Eutreptiaceae indet.",
"Eutreptiaceae indet" : "Eutreptiaceae indet.",
"Eutreptiella" : "Eutreptiella sp.",
"Fitoplancton indet." : "Other phytoplankton",
"Flagellati inf 20 micron" : "Flagellates indet. (ø inf. 20µm)",
"Flagellati inf 20 µm" : "Flagellates indet. (ø inf. 20µm)",
"Flagellati inf a 20 micron" : "Flagellates indet. (ø inf. 20µm)",
"Flagellati inf a 20 µm" : "Flagellates indet. (ø inf. 20µm)",
"Flagellato inf 20 µm" : "Flagellates indet. (ø inf. 20µm)",
"Gymnodiniaceae" : "Gymnodiniaceae indet.", 
"Gymnodiniaceae inf 20 micron" : "Gymnodiniaceae indet. (ø inf. 20µm)",
"Gymnodiniaceae inf 20 µm" : "Gymnodiniaceae indet. (ø inf. 20µm)",
"Gymnodiniaceae sup 20 micron" : "Gymnodiniaceae indet. (ø sup. 20µm)",
"Gymnodiniaceae sup 20 µm" : "Gymnodiniaceae indet. (ø sup. 20µm)",
"Gymnodiniales" : "Gymnodiniales indet.",
"Gymnodiniales (ø inf. 20µm)" : "Gymnodiniales indet. (ø inf. 20µm)",
"Gymnodiniales inf 20 micron" : "Gymnodiniales indet. (ø inf. 20µm)",
"Gymnodiniales inf 20 µm" : "Gymnodiniales indet. (ø inf. 20µm)",
"Gymnodiniales sup 20 micron" : "Gymnodiniales indet. (ø sup. 20µm)",
"Gymnodiniales sup 20 µm" : "Gymnodiniales indet. (ø sup. 20µm)",
"Haptophyta" : "Haptophyta indet.", 
"Ind. sup 20 micron" : "Other phytoplankton (ø sup. 20µm)",
"Indet" : "Other phytoplankton",
"Indeterminati" : "Other phytoplankton",
"Indeterminati forma siluro sup. 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
"Indeterminati inf 20 micron" : "Other phytoplankton (Ø inf. 20µm)",
"Indeterminati inf 20 µm" : "Other phytoplankton (Ø inf. 20µm)",
"Indeterminati inf. 20 micron" : "Other phytoplankton (Ø inf. 20µm)",
"Indeterminati non flag inf. 20 micron" : "Other phytoplankton (Ø inf. 20µm)",
"Indeterminati non flag inf. 20 µm" : "Other phytoplankton (Ø inf. 20µm)",
"Indeterminati non flag sup. 20 micron" : "Other phytoplankton (Ø sup. 20µm)",
"Indeterminati non flag sup. 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
"Indeterminati non flagellati" : "Non flagellates indet. ",
"Indeterminati non flagellati inf 20 micron" : "Non flagellates indet. (Ø inf. 20µm)",
"Indeterminati non flagellati inf 20 µm" : "Non flagellates indet. (Ø inf. 20µm)",
"Indeterminati non flagellati sup 20 micron" : "Non flagellates indet. (Ø sup. 20µm)",
"Indeterminati non flagellati sup 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
"Indeterminati sup 20 micron" : "Other phytoplankton (Ø sup. 20µm)",
"Indeterminati sup 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
"Indeterminati sup a 20 micron" : "Other phytoplankton (Ø sup. 20µm)",
"Indeterminati sup. 20 micron" : "Other phytoplankton (Ø sup. 20µm)",
"Indeterminato inon flagellati sup 20 µm" : "Non flagellates indet. (Ø sup. 20µm)",
"Indeterminato non flagellato inf 20 µm" : "Non flagellates indet. (Ø inf. 20µm)",
"Indeterminato non flagellato sup 20 µm" : "Non flagellates indet. (Ø sup. 20µm)",
"Indeterminato sup 20 µm" : "Other phytoplankton (Ø sup. 20µm)",
"Lessardia elongata > 20 um" : "Lessardia elongata",
"Navicula sp. inf 20 µm" : "Navicula sp. (Ø inf. 20µm)",
"Navicula sp. sup 20 µm" : "Navicula sp. (Ø sup. 20µm)",
"Navicula spp. sup 20 micron" : "Navicula spp. (Ø sup. 20µm)",
"Navicula spp. sup. 20 micron" : "Navicula spp. (Ø sup. 20µm)",
"Naviculaceae indet" : "Naviculaceae indet.",
"Octactis octonaria v. pulchra" : "Octactis octonaria var. pulchra", 
"Oscillatoriales" : "Oscillatoriales indet.",
"Oscillatoriales ind." : "Oscillatoriales indet.",
"Oxytoxum variabile < 20" : "Oxytoxum variabile",
"Oxytoxum variabile < 20 µm" : "Oxytoxum variabile",
"Oxytoxum variabile > 20" : "Oxytoxum variabile",
"Oxytoxum variabile > 20 µm" : "Oxytoxum variabile",
"Pedinellaceae" : "Pedinellaceae indet.",
"Pedinellaceae indet" : "Pedinellaceae indet.",
"Pedinellales" : "Pedinellales indet.", 
"Peridiniales" : "Peridiniales indet.",
"Peridiniales inf 20 micron" : "Peridiniales indet. (Ø inf. 20µm)",
"Peridiniales spp." : "Peridiniales indet.",
"Peridiniales sup. 20 micron" : "Peridiniales indet. (Ø sup. 20µm)", 
"Plagioselmis" : "Plagioselmis sp.", 
"Prasinophyceae" : "Prasinophyceae indet.",
"Prasinophyceae indet" : "Prasinophyceae indet.",
"Prymnesiales" : "Prymnesium indet.",
"Prymnesiophyceae" : "Coccolithophyceae indet.",
"Prymnesiophyceae indet." : "Coccolithophyceae indet.",
"Pseudonitzschia group spp." : "Pseudo-nitzschia spp.", 
"Pseudonitzschia pseudelicatissima" : "Pseudo-nitzschia pseudodelicatissima",
"Pseudonitzschia pseudodelicatissima group" : "Pseudo-nitzschia pseudodelicatissima",
"Pseudonitzschia s." : "Pseudo-nitzschia sp.", 
"Pseudosolenia calcar- avis" : "Pseudosolenia calcar-avis",
"Pyramimonadophyceae" : "Pyramimonadophyceae indet.",
"Raphidophyceae" : "Raphidophyceae indet.",
"Raphidophyceae indt." : "Raphidophyceae indet.",
"Syracosphaera pulchra > 20" : "Syracosphaera pulchra",
"Syracosphaera pulchra > 20um" : "Syracosphaera pulchra",
"Torodinium robustum > 20" : "Torodinium robustum", 
"Cianophyceae indet." : "Cyanophyceae indet.", 
"Coscinodiscophycea indet." : "Coscinodiscophycidae indet.", 
"Pseudonitzschia fraudolenta" : "Pseudo-nitzschia fraudulenta", 
"Pseudonitzschia delicatissima." : "Pseudo-nitzschia delicatissima", 
"A. cf. aculeata" : "Acanthocardia sp.", 
"Rhoicosphenia" : "Rhoicosphenia spp.",
"Protoperidinium" : "Protoperidinium spp.",
"Tetraselmis" : "Tetraselmis spp.", 
"Kinetoplastidea" : "Kinetoplastidea indet.", 
"Gonyaulacaceae" : "Gonyaulacaceae indet.", 
"Mediophyceae" : "Mediophyceae indet.", 
"Lingulodinium polyedrum" : "Lingulodinium polyedra",
"Ditylum brightwelli" : "Ditylum brightwellii",
"Gyrosigma fusiforme" : "Gyrosigma sp.",
"Chattonella subsalsa tipo" : "Chattonella subsalsa",
"Chaetoceros blavyanus" : "Dactyliosolen blavyanus", 
"Thalassiosira weissfl gii" : "Thalassiosira weissflogii",
"Karlodinium glaucum" : "Katodinium glaucum",
"Scyphosphaera elegans" : "Other phytoplankton", 
'Bacillariophyceae bacillariales' : 'Bacillariales', 
'Bacillariophyceae biddulphiales' : "Biddulphiales",
"Coscinodiscus radiates" : "Coscinodiscus radiatus", 
'Lessar elongata' : 'Lessardia elongata', 
'Lessardia elongata > 20 um' : 'Lessardia elongata', 
"Melosira nummulloides" : "Melosira nummuloides", 
"Chaetoceros deblis" : "Chaetoceros debilis", 
"Chaetoceros peruvinus" : "Chaetoceros peruvianus", 
'Indeterminati inf 20 µm' : "Other phytoplankton", 
'Indeterminati sup 20 µm' : "Other phytoplankton (Ø sup. 20µm)", 
'Indeterminati non flagellati' : "Non flagellates indet.",
"Indeterminat inon flagellati sup 20 µm" : "Non flagellates indet. (Ø sup. 20µm)", 
"Diatomee indet." : "Bacillariophyceae indet.",
'Altro_fitoplancton' : "Other phytoplankton", 
'Diatomee' : "Bacillariophyceae indet.", 
'Diatomee  ' : "Bacillariophyceae indet.", 
'Dinoflagellate' : "Dinoflagellata indet.", 
"Prorocentrum gracilis" : "Prorocentrum gracile", 
"Pyrocystis obstusa" : "Pyrocystis obtusa", 
"Protoperidinium steiini" : "Protoperidinium steinii", 
"Bacteriastrum delicatum" : "Bacteriastrum delicatulum",
"Guinardia flaccia" : "Guinardia flaccida", 
"Protoperidinium steini" :  "Protoperidinium steinii",
"Protoperidinium trystilum" : "Protoperidinium tristylum", 
"Skeletonema mediterraneum" : "Skeletonema mediterranea", 
"Ollicola vangoori" : "Ollicola vangoorii", 
"Dactyliosolen puketensis" : "Dactyliosolen phuketensis", 
"Skeletonema pseudocostatus" : "Skeletonema pseudocostatum", 
"Rhodomonas amphiozieria" : "Rhodomonas amphioxeia", 
"Syracosphaera apsteinii" : "Syracolithus schilleri", 
"Protoperidinium stenii" : "Protoperidinium steinii",
"Chaetoceros diersus" : "Chaetoceros diversus", 
"Scyphosphaera pulchra" : "Syracosphaera pulchra", 
"Rabdosphaera claviger" : "Rhabdosphaera clavigera", 
"Syracosphaera apnestii" : "Syracolithus schilleri", 
"Rhabdosphaera claviger" : "Rhabdosphaera clavigera", 
"Kinetoplastidea indet.": "Kinetoplastea indet.",
"Chaetoceros peruvianos" : "Chaetoceros peruvianus", 
"Chaetoceros crvisetus" : "Chaetoceros curvisetus", 
"Dictyocha fbula" : "Dictyocha fibula", 
"Bacteriastrum fureatum" : "Bacteriastrum furcatum", 
"Ceratocorys gourreti" : "Ceratocorys gourretii", 
"Ceratium hirudinella" : "Ceratium hirundinella", 
"Syracolithus dentatus" : "Syracolithus sp.", 
"Gymnodinium vestificii" : "Gymnodinium vestifici", 
"Dactyliosolen antarticus" : "Dactyliosolen antarcticus",
"Gonyaulax monocantha" : "Gonyaulax monacantha",
"Chaetoceros daday" : "Chaetoceros dadayi", 
"Rhabdosphaera stylifer" : "Rhabdosphaera stylifera", 
"Diploneis cabro" : "Diploneis crabro", 
"Pyramimonas grosii" : "Pyramimonas grossii", 
"Chaetoceros laevis" : "Chaetoceros levis", 
"Oxytoxum constrictus" : "Oxytoxum constrictum",
"Nephrophyceae indet" : 'Nephroselmidophyceae indet.'
}

In [19]:
taxa = phyto_abund_raw.loc[:, "Taxon"]
for before, after in partial_corrections.items(): 
    taxa = taxa.apply(lambda name: re.sub(before, after, name))

for before, after in complete_corrections.items(): 
    taxa[taxa == before] = after

# def correct_names(name):
#     new = complete_corrections.get(name) 
#     if new: 
#         return new
#     return name
# taxa = taxa.apply(correct_names)


In [20]:
regex_commands = {
    "(sp\.p\.+|sp\.p\.+|spp\.+|sp\.p\.\.|spp\.\.|sp\.p|spp|sp\. p\.)" : "spp.", 
    "(sp$|sp.2)" : "sp.",
    '(?<=[.,])(?=[^\s])' : ' ',
    #"indet\.*" : "indet.",
    "cf\.*" : "cf."
}
for before, after in regex_commands.items(): 
    taxa = taxa.apply(lambda name: re.sub(before, after, name))

In [21]:
taxa[taxa == "Altro fitoplancton indet. (ø inf. 20µm)"] = "Other phytoplankton (ø inf. 20µm)"
taxa[taxa == "Altro fitoplancton indet. (ø sup. 20µm)"] = "Other phytoplankton (ø sup. 20µm)"
taxa[taxa == "Diatoma s. p."] = "Diatoma sp."
taxa[taxa == "Diatoma s. p. p."] = "Diatoma spp."
taxa[taxa == "Other Phytoplankton"] = "Other phytoplankton"
taxa[taxa == "Other phytoplankton non flagellati sup 20 µm"] = "Other phytoplankton (ø sup. 20µm)"
taxa[taxa == "Other phytoplankton inf 20 µm"] = "Other phytoplankton (ø inf. 20µm)"


In [22]:
back_up = phyto_abund_raw.copy(deep=True)

In [23]:
taxa = taxa.apply(lambda x: re.sub("indet(?!\.)", "indet.", x))

In [25]:
phyto_abund_raw["Taxon"] = taxa

In [26]:
phyto_abund_raw[["Region", "id", "Longitude", "Latitude", "Closest_coast", "Date", "SampleDepth", "Group", "Taxon", "Num_cell_l", "Remarks", "file_name"]].to_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_corrected.csv")
del phyto_abund_raw

## modify dataset 

subsituting in Marche samples with "<120" cell/L with uniform random number 

In [4]:
phyto_abundances = pd.read_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_corrected.csv",index_col=0)

/tmp/ipykernel_804/2115254911.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  phyto_abundances = pd.read_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_corrected.csv",index_col=0)


In [5]:
phyto_abundances.loc[phyto_abundances["Num_cell_l"] == "<120", "Num_cell_l"] = np.vectorize(str)(np.random.randint(10, 120, phyto_abundances.loc[phyto_abundances["Num_cell_l"] == "<120", "Num_cell_l"].size))
phyto_abundances["Num_cell_l"] = phyto_abundances["Num_cell_l"].astype(float)
phyto_abundances.rename(columns={"SampleDepth" : "Sample_depth"}, inplace=True)
phyto_abundances["Date"] = pd.to_datetime(phyto_abundances["Date"], format = '%Y-%m-%d')
phyto_abundances.drop_duplicates(inplace=True)
phyto_abundances = phyto_abundances[phyto_abundances["Num_cell_l"] != 0]

moving double sample to previous or following month when they are temporally close  

In [6]:
cond = (phyto_abundances["Region"] == "Emilia-Romagna") & ((phyto_abundances["Date"] == "2015-10-05") | (phyto_abundances["Date"] == "2015-10-09"))
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2015, 9, 30)
cond = (phyto_abundances["Region"] == "Marche") & ((phyto_abundances["Date"] == "2016-03-08") | (phyto_abundances["Date"] == "2016-03-02"))
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2016, 2, 27)
cond = (phyto_abundances["Region"] == "Marche") & (phyto_abundances["Date"] == "2017-04-04")
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2017, 3, 30)
cond = (phyto_abundances["Region"] == "Marche") & (phyto_abundances["Date"] == "2017-05-02")
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2017, 4, 30)
cond = (phyto_abundances["Region"] == "Veneto") & (phyto_abundances["Date"] == "2016-05-05")
phyto_abundances.loc[cond, "Date"] = datetime.datetime(2016, 4, 30)

grouping together higher cat. taxa 

In [7]:
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("sp\.|cf.\s.*", "spp.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Bacillariophyceae.*", "Bacillariophyceae indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Bacillariophyta.*", "Bacillariophyta indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Chlorodendrophyceae.*", "Chlorophyta indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Chlorophyceae.*", "Chlorophyta indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Coccolithophyceae.*", "Coccolithophyceae indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Dinophyceae.*", "Dinophyceae indet.", regex = False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Euglenophyceae.*", "Euglenophyceae indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Gymnodiniales.*", "Gymnodiniales indet.", regex = True) 
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Gymnodiniaceae.*","Gymnodiniaceae indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Peridiniales.*", "Peridiniales indet.", regex = True)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace(" \(.*\)", "", regex = True, case=False)
phyto_abundances.loc[:, "Taxon"] = phyto_abundances.loc[:, "Taxon"].str.replace("Cyanophyceae indet. filamenti", "Cyanophyceae indet.", regex = False, case=False)

removing species var. cf. f.

In [8]:
taxons = phyto_abundances.loc[phyto_abundances["Taxon"].str.contains("var.", regex=False), "Taxon"].unique()
condition = phyto_abundances["Taxon"].isin(taxons)
phyto_abundances.loc[condition, "Taxon"] = phyto_abundances.loc[condition, "Taxon"].str.replace(" var.*", "",regex=True)
condition = phyto_abundances["Taxon"].str.contains("cf\.|f\.", case=False, regex=True)
phyto_abundances.loc[condition, "Taxon"] = phyto_abundances.loc[condition, "Taxon"].str.replace("Cf\. | f\. .*", "", regex=True)

In [12]:
condition = phyto_abundances["Taxon"].str.endswith(" ")
phyto_abundances.loc[condition, "Taxon"] = phyto_abundances.loc[condition, "Taxon"].apply(lambda x: x.strip()) 

In [14]:
phyto_abundances = phyto_abundances.groupby(["Region", "id", "Longitude", "Latitude", "Closest_coast", "Date", "Sample_depth", "Group", "Taxon"]).agg({"Num_cell_l" : "sum", "file_name" : "first", "Remarks" : "first"}).reset_index()

In [7]:
phyto_abundances.loc[phyto_abundances["Taxon"] == "Prymnesiophyceae indet.", "Taxon"] = "Coccolithophyceae"

In [5]:
phyto_abundances.loc[phyto_abundances["Taxon"].str.contains("indet.", regex = False), "Taxon"] = phyto_abundances.loc[phyto_abundances["Taxon"].str.contains("indet.", regex = False), "Taxon"].apply(lambda x: x.split( " indet.")[0])

In [9]:
phyto_abundances.loc[phyto_abundances["Taxon"] == "Dinoflagellata ", "Taxon"] = "Dinoflagellata incertae sedis"

In [5]:
phyto_abundances.loc[:, "Taxon"] = phyto_abundances["Taxon"].apply(remove_additional_white_spaces)

In [6]:
phyto_abundances.to_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_modified.csv")

In [4]:
phyto_abundances =pd.read_csv(_HOME_ + "/PHD/MSFD/Data/Modulo1/phyto_abund_modified.csv", index_col = 0)